<a href="https://colab.research.google.com/github/mertqara/PopularGameGenres/blob/main/data_merge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Re-merge using only vgsales.csv and metacritic.csv
import pandas as pd

# Load and clean sales data
sales_df = pd.read_csv(vgsales_url)
sales_df.rename(columns={'Name': 'Game', 'Year': 'Release_Year'}, inplace=True)
sales_df['Game'] = sales_df['Game'].str.lower().str.strip()
sales_df['Release_Year'] = pd.to_numeric(sales_df['Release_Year'], errors='coerce')

# Load and clean metacritic data
meta_df = pd.read_csv(metacritic_url)
meta_df.rename(columns={'name': 'Game', 'meta_score': 'MetaScore', 'user_review': 'UserReview'}, inplace=True)
meta_df['Game'] = meta_df['Game'].str.lower().str.strip()
meta_df['Release_Year'] = pd.to_datetime(meta_df['release_date'], errors='coerce').dt.year
meta_df = meta_df[['Game', 'Release_Year', 'MetaScore', 'UserReview']]

# Merge sales and metacritic data
merged_simple_df = pd.merge(sales_df, meta_df, on=['Game', 'Release_Year'], how='outer')

# Clean non-numeric scores
merged_simple_df['MetaScore'] = pd.to_numeric(merged_simple_df['MetaScore'], errors='coerce')
merged_simple_df['UserReview'] = pd.to_numeric(merged_simple_df['UserReview'], errors='coerce')

# Remove future years and games with 0 global sales
merged_simple_df = merged_simple_df[merged_simple_df['Release_Year'] <= 2024]
merged_simple_df = merged_simple_df[merged_simple_df['Global_Sales'] > 0]

# Deduplication:
# Group by unique Game + Platform + Release_Year + Genre + Publisher
# SUM the sales and AVERAGE the scores
merged_deduped = merged_simple_df.groupby(
    ['Game', 'Platform', 'Release_Year', 'Genre', 'Publisher'],
    as_index=False
).agg({
    'NA_Sales': 'sum',
    'EU_Sales': 'sum',
    'JP_Sales': 'sum',
    'Other_Sales': 'sum',
    'Global_Sales': 'sum',
    'MetaScore': 'mean',
    'UserReview': 'mean'
})

# Display top rows
display(merged_deduped.head(20))

# Save for future analysis
merged_vgsales_metacritic_only = merged_deduped

# Save the cleaned dataset as CSV
merged_vgsales_metacritic_only.to_csv("cleaned_video_game_data.csv", index=False)


,Game,Platform,Release_Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,MetaScore,UserReview
0,'98 koshien,PS,1998.0,Sports,Magical Company,0.15,0.10,0.12,0.03,0.41,NaN,NaN
1,.hack//g.u. vol.1//rebirth,PS2,2006.0,Role-Playing,Namco Bandai Games,0.00,0.00,0.17,0.00,0.17,NaN,NaN
2,.hack//g.u. vol.2//reminisce,PS2,2006.0,Role-Playing,Namco Bandai Games,0.11,0.09,0.00,0.03,0.23,NaN,NaN
3,.hack//g.u. vol.2//reminisce (jp sales),PS2,2006.0,Role-Playing,Namco Bandai Games,0.00,0.00,0.16,0.00,0.16,NaN,NaN
4,.hack//g.u. vol.3//redemption,PS2,2007.0,Role-Playing,Namco Bandai Games,0.00,0.00,0.17,0.00,0.17,NaN,NaN
5,.hack//infection part 1,PS2,2002.0,Role-Playing,Atari,0.49,0.38,0.26,0.13,1.27,NaN,NaN
6,.hack//link,PSP,2010.0,Role-Playing,Namco Bandai Games,0.00,0.00,0.14,0.00,0.14,NaN,NaN
7,.hack//mutation part 2,PS2,2002.0,Role-Playing,Atari,0.23,0.18,0.20,0.06,0.68,NaN,NaN
8,.hack//outbreak part 3,PS2,2002.0,Role-Playing,Atari,0.14,0.11,0.17,0.04,0.46,NaN,NaN
9,.hack//quarantine part 4: the final chapter,PS2,2003.0,Role-Playing,Atari,0.09,0.07,0.00,0.02,0.18,NaN,NaN
